# The Process of Model Building

1-Anaconda — Anaconda is a free and open source distribution of the Python and R programming languages for data science and machine learning related applications, that aims to simplify package management and deployment.
You can download it from the link below according to your system https://www.anaconda.com/download/

2-Keras — Keras is an open source neural network library written in Python.
Activate Tensorflow env and install keras using ‘pip install keras’.

3-CNN — Convolution Neural network , a class of deep, feed-forward artificial neural networks, most commonly applied to analyzing visual imagery.
Here is a very good explanation to what it actually is -https://ujjwalkarn.me/2016/08/11/intuitive-explanation-convnets/


#  Import the libraries we’ll need during our model building phase.

In [1]:
# (1) Importing dependency
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import numpy as np
np.random.seed(1000)

Using TensorFlow backend.


# AlexNet Architecture

In [2]:
# Create a sequential model
model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(filters=96, input_shape=(224,224,3), kernel_size=(11,11),\
 strides=(4,4), padding='valid'))
model.add(Activation('relu'))
# Pooling 
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation before passing it to the next layer
model.add(BatchNormalization())

# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())

# 3rd Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())

# Passing it to a dense layer
model.add(Flatten())
# 1st Dense Layer
model.add(Dense(4096, input_shape=(224*224*3,)))
model.add(Activation('relu'))
# Add Dropout to prevent overfitting
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# 2nd Dense Layer
model.add(Dense(4096))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# 3rd Dense Layer
model.add(Dense(1000))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# Output Layer
model.add(Dense(23))
model.add(Activation('softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 54, 54, 96)        34944     
_________________________________________________________________
activation_1 (Activation)    (None, 54, 54, 96)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 27, 27, 96)        384       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 17, 17, 256)       2973952   
_________________________________________________________________
activation_2 (Activation)    (None, 17, 17, 256)       0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 256)         0         
__________

In [3]:
train_dir = 'Dataset/train'
test_dir = 'Dataset/final'
validation_dir = 'Dataset/test'
small_test_dir = 'Dataset/small'

In [4]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=40,
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    shear_range=0.2,
                    zoom_range=0.2)

In [5]:
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
                    train_dir,
                    target_size=(224, 224),
                    batch_size=16,
                    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
                        validation_dir,
                        target_size=(224, 224),
                        batch_size=16,
                        class_mode='categorical')
datagen = ImageDataGenerator(rescale=1./255)
test=datagen.flow_from_directory(
        test_dir,
        target_size=(224, 224),
        batch_size=16,
       class_mode='categorical')

Found 35247 images belonging to 23 classes.
Found 8679 images belonging to 23 classes.
Found 1150 images belonging to 23 classes.


In [6]:

from keras import optimizers
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy', optimizer='adam',
                  metrics=['accuracy'])

In [7]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=20, verbose=0,
                                  mode='min')


ckpt = ModelCheckpoint('alexNet_model.hdf5', save_best_only=True,
                           monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                       patience=7, verbose=1, epsilon=1e-4,
                                       mode='min')

In [ ]:
history = model.fit_generator(
                train_generator,
                steps_per_epoch=50,
                epochs=50,
                callbacks=[earlyStopping, ckpt, reduce_lr_loss],
                validation_data=validation_generator,
                validation_steps=20)

Epoch 1/50


In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [14]:
from keras.models import load_model
model = load_model('small_last4.h5')

In [15]:
small=datagen.flow_from_directory(
        small_test_dir,
        target_size=(224, 224),
        batch_size=1,
       class_mode='categorical')

Found 1 images belonging to 23 classes.


In [21]:
#test model accurcy in new images 
test_accuracy=model.evaluate_generator(test,1150)

In [19]:
#x=model.evaluate_generator(small, steps=1, max_queue_size=1)

In [22]:
print("Accuracy = ", test_accuracy[1]*100)

Accuracy =  96.5160587857


In [11]:
print("Accuracy = ", test_accuracy[1]*100)

Accuracy =  96.5215024483


In [12]:
print(test_accuracy)

[0.12430336783875177, 0.96521502448348295]


In [20]:
print("Accuracy = ", x[1]*100)

Accuracy =  0.0
